In [1]:
import dlt
from dlt.destinations.filesystem import filesystem
import pyarrow.parquet as pq
import requests
import pandas as pd
import os
import tempfile
import pyarrow.parquet as pq
from minio import Minio
from io import BytesIO

In [3]:
import dlt
print(dlt.secrets["endpoint_url"])

ConfigFieldMissingException: Following fields are missing: ['endpoint_url'] in configuration with spec Any
	for field "endpoint_url" config providers and keys were tried in following order:
		In Environment Variables key ENDPOINT_URL was not found.
Please refer to https://dlthub.com/docs/general-usage/credentials for more information


In [ ]:
# Configuración de la URL
URL = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-01.parquet"

@dlt.resource(name="yellow_trip_data", write_disposition="replace")
def yellow_trip_data():
    try:
        print("Descargando archivo parquet...")
        response = requests.get(URL, stream=True)
        response.raise_for_status()

        with tempfile.NamedTemporaryFile(suffix=".parquet") as tmp:
            tmp.write(response.content)
            tmp.flush()

            # Leer en formato Arrow
            table = pq.read_table(tmp.name)
            print(f"Datos cargados en Arrow: {table.num_rows} filas, {table.num_columns} columnas")
            yield table

    except Exception as e:
        print(f"Error al descargar o procesar datos: {e}")
        raise

def main():
    try:
        # Configuración de MinIO
        minio_endpoint = os.getenv('MINIO_ENDPOINT', 'minio:9000')  # Cambiado a 'minio' para Docker
        bucket_name = os.getenv('BUCKET_NAME', 'warehouse')         # Usando el bucket existente
        access_key = os.getenv('MINIO_ACCESS_KEY', 'admin')
        secret_key = os.getenv('MINIO_SECRET_KEY', 'password')
        
        # Construir la URL de MinIO (S3 compatible)
        minio_url = f"s3://{bucket_name}?endpoint=http://{minio_endpoint}&access_key={access_key}&secret_key={secret_key}&region=us-east-1"
        print(f"MinIO URL: {minio_url}")
        print("pipeline configured")
        # Configurar pipeline
        pipeline = dlt.pipeline(
            pipeline_name="yellow_trip_data_pipeline",
            destination='filesystem',
            dataset_name="yellow_trip",
            full_refresh=True,
        )
        
        # Ejecutar el pipeline
        print("Iniciando ingesta de datos...")
        load_info = pipeline.run(
            yellow_trip_data(),
            loader_file_format="parquet",
            #destination=dlt.destinations.filesystem(bucket_url=minio_url)
        )
        
        print("Pipeline ejecutado exitosamente!")
        print(f"Load info: {load_info}")
        
    except Exception as e:
        print(f"Error en el pipeline: {e}")
        raise

if __name__ == "__main__":
    main()

MinIO URL: s3://warehouse?endpoint=http://minio:9000&access_key=admin&secret_key=password&region=us-east-1
pipeline configured
Error en el pipeline: Destination s3 is not one of the standard dlt destinations


UnknownDestinationModule: Destination s3 is not one of the standard dlt destinations